In [1]:
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = "retina"

from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))


/tmp/ipykernel_17797/945712042.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import pandas as pd
import os
import argparse
from typing import Any, Callable, Sequence
from pathlib import Path
from PIL import Image
import matplotlib.image as mpimg

import jax
import jax.random as jr
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax.flatten_util import ravel_pytree
import numpy as np
import matplotlib.pyplot as plt
from flax import linen as nn


from bong.util import run_rebayes_algorithm, gaussian_kl_div, MLP
from bong.src import bbb, blr, bog, bong, experiment_utils
#from bong.agents import AGENT_NAMES




Matplotlib created a temporary cache directory at /tmp/matplotlib-yt_twxu7 because the default path (/teamspace/studios/this_studio/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
2024-05-19 16:18:07.698274: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [19]:
print(1 != 2)

True


In [17]:
results_dir = '/teamspace/studios/this_studio/jobs/sarcos-mlp5'

df = pd.read_csv(f'{results_dir}/summary.csv')
df.head(n=20)

,jobname,agent_name,agent_full_name,model_name,elapsed,summary
0,sarcos-mlp5-00,bong-fc-lin,bong-fc-R0-Lin1-EF99-MC99-I1-LR0_0,mlp_10_1[P=241],1.013431,"NLL-PI: 6.9862, NLPD-MC: 7.2081"
1,sarcos-mlp5-01,bong-fc-lin,bong-fc-R0-Lin1-EF99-MC99-I1-LR0_0,mlp_10_1[P=241],1.038297,"NLL-PI: 6.9912, NLPD-MC: 7.0480"


In [18]:

results_dir = '/teamspace/studios/this_studio/jobs/sarcos-mlp2'

df = pd.read_csv(f'{results_dir}/jobs.csv')
#df = pd.read_csv(f'{results_dir}/summary.csv')
df.head(n=20)

,jobname,algo,param,lin,dlr_rank,ef,nsample,niter,lr,model_str,dataset,data_dim,dgp_type,dgp_str,model_type,ntrain
0,sarcos-mlp2-00,bong,dlr,0,10,1,100,99,99.0000,10_1,sarcos,0,na,na,mlp,1000
1,sarcos-mlp2-01,bong,dlr,1,10,99,99,99,99.0000,10_1,sarcos,0,na,na,mlp,1000
2,sarcos-mlp2-02,bog,dlr,0,10,1,100,99,0.0001,10_1,sarcos,0,na,na,mlp,1000
3,sarcos-mlp2-03,bog,dlr,0,10,1,100,99,0.0010,10_1,sarcos,0,na,na,mlp,1000
4,sarcos-mlp2-04,bog,dlr,0,10,1,100,99,0.0100,10_1,sarcos,0,na,na,mlp,1000
5,sarcos-mlp2-05,bog,dlr,1,10,99,99,99,0.0001,10_1,sarcos,0,na,na,mlp,1000
6,sarcos-mlp2-06,bog,dlr,1,10,99,99,99,0.0010,10_1,sarcos,0,na,na,mlp,1000
7,sarcos-mlp2-07,bog,dlr,1,10,99,99,99,0.0100,10_1,sarcos,0,na,na,mlp,1000
8,sarcos-mlp2-08,blr,dlr,0,10,1,100,1,0.0001,10_1,sarcos,0,na,na,mlp,1000
9,sarcos-mlp2-09,blr,dlr,0,10,1,100,10,0.0001,10_1,sarcos,0,na,na,mlp,1000


In [10]:

results_dir = '/teamspace/studios/this_studio/jobs/sarcos-mlp2'

df = pd.read_csv(f'{results_dir}/jobs_with_eval.csv')
#df = pd.read_csv(f'{results_dir}/summary.csv')

df.head(n=20)




,jobname,algo,param,lin,dlr_rank,ef,nsample,niter,lr,nlpd_te_final,nlpd_te_mid,nlpd_val_final,nlpd_val_mid
0,sarcos-mlp2-00,bong,fc,0,99,0,100,99,99.0000,1.000000e+20,1.000000e+20,1.000000e+20,1.000000e+20
1,sarcos-mlp2-01,bong,fc,0,99,1,100,99,99.0000,5.226648e+00,5.693078e+00,5.855769e+00,6.424086e+00
2,sarcos-mlp2-02,bong,fc,1,99,99,99,99,99.0000,3.513717e+00,3.941316e+00,3.531262e+00,3.878591e+00
3,sarcos-mlp2-03,bog,fc,0,99,0,100,99,0.0001,6.844352e+00,6.849592e+00,7.665295e+00,7.705179e+00
4,sarcos-mlp2-04,bog,fc,0,99,1,100,99,0.0001,6.844233e+00,6.849668e+00,7.665149e+00,7.705274e+00
5,sarcos-mlp2-05,bog,fc,0,99,0,100,99,0.0010,6.763352e+00,6.815027e+00,7.566556e+00,7.660434e+00
6,sarcos-mlp2-06,bog,fc,0,99,1,100,99,0.0010,6.763540e+00,6.815134e+00,7.566897e+00,7.660597e+00
7,sarcos-mlp2-07,bog,fc,0,99,0,100,99,0.0100,4.751331e+00,4.749788e+00,4.781475e+00,4.905397e+00
8,sarcos-mlp2-08,bog,fc,0,99,1,100,99,0.0100,4.641179e+00,4.637194e+00,4.667538e+00,4.712858e+00
9,sarcos-mlp2-09,bog,fc,1,99,99,99,99,0.0001,6.846887e+00,6.850929e+00,7.668380e+00,7.706919e+00


In [ ]:
#condition = (df['minscore'] != df[args.metric])
condition = (df['lin']==1) | (df['ef']==0)
indices_to_drop = df[condition].index
df_filtered = df.drop(indices_to_drop)

df_filtered.head(n=20)

In [5]:
D = 22
mlp1 = [(D+1)*n + (n+1)*1 for n in [10]] # 1 layer
mlp2 = [(D+1)*n + (n+1)*n + (n+1)*1 for n in [20]] # 2 layer MLP

print(mlp1, mlp2)

[241] [901]


In [8]:
import os
import argparse
import os
import itertools
import pandas as pd
from pathlib import Path
import os
import datetime

def list_subdirectories(directory):
    return [name for name in os.listdir(directory)
            if os.path.isdir(os.path.join(directory, name))]

src = '/teamspace/jobs'
dst = '/teamspace/studios/this_studio/jobs'
dirs = list_subdirectories(src)
for old_name in dirs:
    parts = old_name.split('-')
    num = parts[2]
    new_name = f'expt_timing2-{num}'
    dst_path = Path(dst, new_name, 'work')
    print(f'\n Creating {str(dst_path)}')
    dst_path.mkdir(parents=True, exist_ok=True)

    #cmd = f'mv {directory}/{old_name} {directory}/{new_name}'
    cmd = f'sudo cp -r {src}/{old_name}/work/*.* {dst}/{new_name}/work'
    print(cmd)
    os.system(cmd)


 Creating /teamspace/studios/this_studio/jobs/expt_timing2-13/work
sudo cp -r /teamspace/jobs/expt-timing2-13/work/*.* /teamspace/studios/this_studio/jobs/expt_timing2-13/work

 Creating /teamspace/studios/this_studio/jobs/expt_timing2-10/work
sudo cp -r /teamspace/jobs/expt-timing2-10/work/*.* /teamspace/studios/this_studio/jobs/expt_timing2-10/work

 Creating /teamspace/studios/this_studio/jobs/expt_timing2-02/work
sudo cp -r /teamspace/jobs/expt-timing2-02/work/*.* /teamspace/studios/this_studio/jobs/expt_timing2-02/work

 Creating /teamspace/studios/this_studio/jobs/expt_timing2-00/work
sudo cp -r /teamspace/jobs/expt-timing2-00/work/*.* /teamspace/studios/this_studio/jobs/expt_timing2-00/work

 Creating /teamspace/studios/this_studio/jobs/expt_timing2-15/work
sudo cp -r /teamspace/jobs/expt-timing2-15/work/*.* /teamspace/studios/this_studio/jobs/expt_timing2-15/work

 Creating /teamspace/studios/this_studio/jobs/expt_timing2-41/work
sudo cp -r /teamspace/jobs/expt-timing2-41/work

NameError: name 'keys' is not defined